### Librerias necesarias con Anaconda:
conda install -c conda-forge google-api-python-client

conda install -c conda-forge googletrans 

### Documentación de perspective, sobre todo ver la lista de modelos. Si usamos Perspective las categorías de la investigación debería adecuarse a estos modelos
https://github.com/conversationai/perspectiveapi/blob/master/api_reference.md

In [1]:
import config

In [2]:
#ejemplos de algunos comments de Clarin
texto = "Y LAS RATAS KK SIGUEN PENSANDO QUE EL KIRSCHNERISMO VA A VOLVER........COSTARA MUCHISIMO SALIR DE ESTO, PERO LOS KK NUNCA MAS...."
texto2= "che clarin dejalo tranquilo al ple0tudo de gamurra no lo hagas laburar tanto jajajajaja"
texto3= "CLARIN UN DIARIO INMUNDO, IGUAL QUE ESTE GOBIERNO\
 UN DIARIO QUE LOS NEGROS IGNORANTES DE ESTE PAÍS UTILIZAN PARA LIMPIARSE EL CUULO...\
 UN DIARIO QUE EL GRUPO CLARIN USA PARA EXTORSIONAR Y COIMEAR A LOS GOBIERNOS Y DESPUÉS HACER NEGOCIOS\
 CON UNA PAUTA MILLONARÍA EN DÓLARES Y SESENTA NICK'S TRUCHOS SE ENCARGAN DE LINCHAR A LOS CIUDADANOS QUE LES MOLESTAN...\
                             DECIME EN QUE NACIÓN DEL MUNDO... \
                       EN QUÉ PAIS DE AFRICA OCURRE ALGO IGUAL"
texto4 = 'QUE LE paso a Macri? Nadie invierte 150dolares para perder. que alguien saque la cuenta si conviene vender y poner a interese en en BNACION A VER si conviene o no....que desilusion!'

### En muchos casa los "comentaristas" (trolls) usan entrategias para zafar la censura, habría que ver como preprocesar esto, ejemplos:

Solo lei la " Araña" y me imagine hablaba de la c0ncha p.uta de Cristina Fernandez de Kirchner

che clarin dejalo tranquilo al ple0tudo de gamurra no lo hagas laburar tanto jajajajaja


In [3]:
#Para el caso de querer traducir:

#from googletrans import Translator
#translator = Translator()
#traduccion = translator.translate(texto3).text
#print(traduccion)

In [4]:
from googleapiclient import discovery
import json

API_KEY= config.perspectiveAPI_KEY #Toni

service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)

Modulos que funcionan en español:
TOXICITY_EXPERIMENTAL
SEVERE_TOXICITY_EXPERIMENTAL
IDENTITY_ATTACK_EXPERIMENTAL
INSULT_EXPERIMENTAL
PROFANITY_EXPERIMENTAL
THREAT_EXPERIMENTAL

In [6]:
analyze_request = {
  'comment': { 'text': texto2 },
  'requestedAttributes': {'TOXICITY_EXPERIMENTAL': {}, 
                          'SEVERE_TOXICITY_EXPERIMENTAL':{},
                          'IDENTITY_ATTACK_EXPERIMENTAL':{},
                          'PROFANITY_EXPERIMENTAL':{},
                          'THREAT_EXPERIMENTAL':{},
                          'INSULT_EXPERIMENTAL':{}
                         }
}

response = service.comments().analyze(body=analyze_request).execute()

print (json.dumps(response, indent=2))

{
  "attributeScores": {
    "SEVERE_TOXICITY_EXPERIMENTAL": {
      "spanScores": [
        {
          "begin": 0,
          "end": 87,
          "score": {
            "value": 0.12493451,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.12493451,
        "type": "PROBABILITY"
      }
    },
    "TOXICITY_EXPERIMENTAL": {
      "spanScores": [
        {
          "begin": 0,
          "end": 87,
          "score": {
            "value": 0.19863188,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.19863188,
        "type": "PROBABILITY"
      }
    }
  },
  "languages": [
    "es"
  ]
}


In [9]:
analyze_request = {
  'comment': { 'text': texto4 },
  'requestedAttributes': {'TOXICITY_EXPERIMENTAL': {}, 
                          'SEVERE_TOXICITY_EXPERIMENTAL':{},
                          'PROFANITY_EXPERIMENTAL':{},
                          'THREAT_EXPERIMENTAL':{},
                          'INSULT_EXPERIMENTAL':{}
                         }
}

response = service.comments().analyze(body=analyze_request).execute()

print (json.dumps(response, indent=2))

{
  "attributeScores": {
    "SEVERE_TOXICITY_EXPERIMENTAL": {
      "spanScores": [
        {
          "begin": 0,
          "end": 181,
          "score": {
            "value": 0.085778356,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.085778356,
        "type": "PROBABILITY"
      }
    },
    "TOXICITY_EXPERIMENTAL": {
      "spanScores": [
        {
          "begin": 0,
          "end": 181,
          "score": {
            "value": 0.13242681,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.13242681,
        "type": "PROBABILITY"
      }
    }
  },
  "languages": [
    "es"
  ]
}


In [14]:
response['attributeScores']['TOXICITY_EXPERIMENTAL']['summaryScore']['value']

{'spanScores': [{'begin': 0,
   'end': 181,
   'score': {'type': 'PROBABILITY', 'value': 0.13242681}}],
 'summaryScore': {'type': 'PROBABILITY', 'value': 0.13242681}}

In [17]:
response['attributeScores']['SEVERE_TOXICITY_EXPERIMENTAL']['summaryScore']['value']

0.085778356

In [20]:
import pandas as pd
df = pd.read_csv('./data/hugo-moyano-pedido-detencion-hijo.csv', index_col= 0).dropna().reset_index()

In [76]:
import numpy as np
def analizar(comentarios):
    resultados = []
    for comentario in comentarios:
        resultado = {}
        try:
            analyze_request = {
                            'comment': { 'text': comentario },
                            'requestedAttributes': {'TOXICITY_EXPERIMENTAL': {}, 
                                                      'SEVERE_TOXICITY_EXPERIMENTAL':{}}}
            response = service.comments().analyze(body=analyze_request).execute()
            
            resultado['comment'] = comentario
            resultado['toxicity'] =response['attributeScores']['TOXICITY_EXPERIMENTAL']['summaryScore']['value']
            resultado['severe_toxicity'] = response['attributeScores']['SEVERE_TOXICITY_EXPERIMENTAL']['summaryScore']['value']
            resultados.append(resultado)
        except:
            resultado['comment'] = comentario
            resultado['toxicity'] = np.nan
            resultado['severe_toxicity'] = np.nan
            resultados.append(resultado)
    return resultados     

In [77]:
from joblib import Parallel, delayed

In [78]:
lista_comentarios =np.array_split(df.comments, 4)

In [79]:
%%time
results = Parallel(n_jobs=-1)(delayed(analizar)(comentarios) for comentarios in lista_comentarios)

Wall time: 41.3 s


In [84]:
results = results[0] +results[1] +results[2] +results[3]

In [85]:
df_comentarios = pd.DataFrame(results)

In [88]:
df_comentarios.to_csv('./data/hugo moyano pedido detencion hijo Perspective.csv')